### jai shree ram 

In [5]:
print('Dinesh')

Dinesh


In [13]:
import re 
import numpy as np 
import pandas as pd  
from pprint import pprint 
import gensim 
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess 
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

### Load the dataset 

In [14]:
import gensim

In [15]:
 # NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [16]:
df  = pd.read_csv('preprocessed_final.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed_final.csv'

In [16]:
print(f'List of columns : {df.columns} \n')
print(f'shape of dataset : {df.shape} \n')
print(f'Data type of dataset : {dict(df.dtypes)} \n')

NameError: name 'df' is not defined

### check  unique label in data 

In [ ]:
df.Label.unique()

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype=object)

### remove email and newline  characters 

In [ ]:
## Convert to list 
data = df.Text.values.tolist()

## Remove Emails 
data  =  [re.sub('\S*@S\*\s','',sent)for sent in data]

## remove new line character 
data = [re.sub('\s+',' ',sent) for sent in data]

## Removes distracting single quotes 

data = [re.sub("\'","",sent) for sent in data]

pprint(data[:1])

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dinesh jangir\AppData\Local\Temp\ipykernel_12184\306511511.py:5: SyntaxWarning: invalid escape sequence '\S'
  data  =  [re.sub('\S*@S\*\s','',sent)for sent in data]
C:\Users\dinesh jangir\AppData\Local\Temp\ipykernel_12184\306511511.py:8: SyntaxWarning: invalid escape sequence '\s'
  data = [re.sub('\s+',' ',sent) for sent in data]


['From: mathew <mathew@mantis.co.uk> Subject: Alt.Atheism FAQ: Atheist '
 'Resources Archive-name: atheism/resources Alt-atheism-archive-name: '
 'resources Last-modified: 11 December 1992 Version: 1.0 Atheist Resources '
 'Addresses of Atheist Organizations USA FREEDOM FROM RELIGION FOUNDATION '
 'Darwin fish bumper stickers and assorted other atheist paraphernalia are '
 'available from the Freedom From Religion Foundation in the US. Write to: '
 'FFRF, P.O. Box 750, Madison, WI 53701. Telephone: (608) 256-8900 EVOLUTION '
 'DESIGNS Evolution Designs sell the "Darwin fish". Its a fish symbol, like '
 'the ones Christians stick on their cars, but with feet and the word "Darwin" '
 'written inside. The deluxe moulded 3D plastic fish is $4.95 postpaid in the '
 'US. Write to: Evolution Designs, 7119 Laurel Canyon #4, North Hollywood, CA '
 '91605. People in the San Francisco Bay area can get Darwin Fish from Lynn '
 'Gold -- try mailing <figmo@netcom.com>. For net people who go to Lynn 

In [ ]:
def sent_to_word(sentances):
    for sentance in sentances:
        yield(gensim.utils.simple_preprocess(str(sentance),deacc=True))
        
        
data_words = list(sent_to_word(data))
print(data[:1])

['From: mathew <mathew@mantis.co.uk> Subject: Alt.Atheism FAQ: Atheist Resources Archive-name: atheism/resources Alt-atheism-archive-name: resources Last-modified: 11 December 1992 Version: 1.0 Atheist Resources Addresses of Atheist Organizations USA FREEDOM FROM RELIGION FOUNDATION Darwin fish bumper stickers and assorted other atheist paraphernalia are available from the Freedom From Religion Foundation in the US. Write to: FFRF, P.O. Box 750, Madison, WI 53701. Telephone: (608) 256-8900 EVOLUTION DESIGNS Evolution Designs sell the "Darwin fish". Its a fish symbol, like the ones Christians stick on their cars, but with feet and the word "Darwin" written inside. The deluxe moulded 3D plastic fish is $4.95 postpaid in the US. Write to: Evolution Designs, 7119 Laurel Canyon #4, North Hollywood, CA 91605. People in the San Francisco Bay area can get Darwin Fish from Lynn Gold -- try mailing <figmo@netcom.com>. For net people who go to Lynn directly, the price is $4.95 per fish. AMERICAN 

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'mathew_mathew_mantis_co', 'uk', 'subject', 'alt_atheism_faq', 'atheist', 'resources', 'archive_name', 'atheism', 'resources', 'alt_atheism', 'archive_name', 'resources', 'last_modified', 'december', 'version', 'atheist', 'resources', 'addresses', 'of', 'atheist', 'organizations', 'usa', 'freedom', 'from', 'religion', 'foundation', 'darwin_fish', 'bumper_stickers', 'and', 'assorted', 'other', 'atheist', 'paraphernalia', 'are', 'available', 'from', 'the', 'freedom', 'from', 'religion', 'foundation', 'in', 'the', 'us', 'write', 'to', 'ffrf', 'box', 'madison_wi', 'telephone', 'evolution_designs', 'evolution_designs', 'sell', 'the', 'darwin_fish', 'its', 'fish', 'symbol', 'like', 'the', 'ones', 'christians', 'stick', 'on', 'their', 'cars', 'but', 'with', 'feet', 'and', 'the', 'word', 'darwin', 'written', 'inside', 'the', 'deluxe', 'moulded', 'plastic', 'fish', 'is', 'postpaid', 'in', 'the', 'us', 'write', 'to', 'evolution_designs', 'laurel', 'canyon', 'north', 'hollywood', 'ca', '

In [ ]:
 ##Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words_nostops[0][:10]  

['mathew',
 'mathew',
 'mantis',
 'co',
 'uk',
 'alt',
 'atheism',
 'faq',
 'atheist',
 'resources']

In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams[0][:10]



['mathew_mathew',
 'mantis_co',
 'uk',
 'alt_atheism',
 'faq',
 'atheist',
 'resources',
 'archive_name',
 'atheism',
 'resources']

In [ ]:

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['mantis_co', 'alt_atheism', 'faq', 'atheist', 'resource', 'archive_name', 'atheism', 'resource', 'alt_atheism', 'archive_name', 'resource', 'last_modifie', 'version', 'atheist', 'resource', 'address', 'organization', 'religion', 'foundation', 'assort', 'atheist', 'available', 'freedom', 'religion', 'foundation', 'write', 'ffrf', 'telephone', 'evolution', 'design', 'evolution', 'design', 'sell', 'darwin_fish', 'fish', 'symbol', 'one', 'stick', 'car', 'foot', 'word', 'write', 'inside', 'deluxe', 'mould', 'plastic', 'fish', 'postpaid', 'write', 'evolution', 'design', 'canyon', 'north', 'hollywood', 'people', 'san_francisco', 'get', 'lynn', 'gold', 'try', 'mail', 'figmo', 'com', 'net', 'people', 'go', 'directly', 'price', 'fish', 'publish', 'various', 'atheist', 'book', 'critique', 'list', 'book', 'handbook', 'isbn', 'contradiction', 'absurditie', 'atrocity', 'immorality', 'contain', 'contradict', 'base', 'write', 'cameron', 'road', 'austin_tx', 'telephone', 'fax', 'prometheus_book', 'se

In [ ]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 3), (12, 3), (13, 1), (14, 1), (15, 1), (16, 1), (17, 3), (18, 2), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 5), (26, 1), (27, 1), (28, 1), (29, 3), (30, 7), (31, 6), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 4), (45, 1), (46, 2), (47, 2), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 8), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 3), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 4), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1)

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]][0][:10]

[('abortion', 1),
 ('absurditie', 1),
 ('accompany', 1),
 ('account', 1),
 ('address', 3),
 ('adulterie', 1),
 ('aesthetic', 1),
 ('african_american', 1),
 ('agnostic', 1),
 ('alien', 1)]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.353*"law" + 0.093*"break" + 0.089*"private" + 0.072*"medium" + '
  '0.058*"company" + 0.024*"middle" + 0.021*"protection" + 0.019*"profit" + '
  '0.017*"million" + 0.016*"massive"'),
 (1,
  '0.133*"gun" + 0.071*"theory" + 0.060*"society" + 0.043*"freedom" + '
  '0.038*"judge" + 0.038*"tax" + 0.034*"legal" + 0.032*"vote" + 0.029*"car" + '
  '0.029*"seek"'),
 (2,
  '0.021*"group" + 0.020*"system" + 0.018*"support" + 0.017*"include" + '
  '0.014*"base" + 0.014*"use" + 0.013*"provide" + 0.013*"public" + '
  '0.013*"create" + 0.012*"member"'),
 (3,
  '0.179*"morning" + 0.036*"resort" + 0.024*"gene" + 0.000*"ms_myer" + '
  '0.000*"president" + 0.000*"yesterday" + 0.000*"armenian" + 0.000*"compound" '
  '+ 0.000*"dee_dee" + 0.000*"brief"'),
 (4,
  '0.014*"also" + 0.014*"time" + 0.012*"man" + 0.012*"find" + 0.011*"first" + '
  '0.011*"year" + 0.011*"call" + 0.011*"work" + 0.010*"look" + '
  '0.009*"question"'),
 (5,
  '0.066*"ancient" + 0.065*"measure" + 0.047*"image" + 0.047*"reques

## Evaluate topic models

##### Compute model Perplexity and Coherence score Coherence score and perplexity provide a convinent way to measure how good a given topic model is

##### Note:

##### Lower the perplexity better the model.
##### Higher the topic coherence, the topic is more human interpretable.

In [5]:
 #Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

NameError: name 'lda_model' is not defined

In [6]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

NameError: name 'lda_model' is not defined